# Lanternfish

The analysis that follows pertains to the sixth day of the [Python Problem-Solving Bootcamp](https://mathspp.com/pythonbootcamp).

In the analysis that follows you may be confronted with code that you do not understand, especially as you reach the end of the explanation of each part.

If you find functions that you didn't know before, remember to [check the docs](https://docs.python.org/3/) for those functions and play around with them in the REPL.
This is written to be increasing in difficulty (within each part of the problem), so it is understandable if it gets harder as you keep reading.
That's perfectly fine, you don't have to understand everything _right now_, especially because I can't know for sure what _your level_ is.

## Part 1 problem statement

(From [Advent of Code 2021, day 6](https://adventofcode.com/2021/day/6))

The sea floor is getting steeper. Maybe the sleigh keys got carried this way?

A massive school of glowing [lanternfish](https://en.wikipedia.org/wiki/Lanternfish) swims past. They must spawn quickly to reach such large numbers - maybe _exponentially_ quickly? You should model their growth rate to be sure.

Although you know nothing about this specific species of lanternfish, you make some guesses about their attributes. Surely, each lanternfish creates a new lanternfish once every _7_ days.

However, this process isn't necessarily synchronized between every lanternfish - one lanternfish might have 2 days left until it creates another lanternfish, while another might have 4. So, you can model each fish as a single number that represents _the number of days until it creates a new lanternfish_.

Furthermore, you reason, a _new_ lanternfish would surely need slightly longer before it's capable of producing more lanternfish: two more days for its first cycle.

So, suppose you have a lanternfish with an internal timer value of `3`:

-   After one day, its internal timer would become `2`.
-   After another day, its internal timer would become `1`.
-   After another day, its internal timer would become `0`.
-   After another day, its internal timer would reset to `6`, and it would create a _new_ lanternfish with an internal timer of `8`.
-   After another day, the first lanternfish would have an internal timer of `5`, and the second lanternfish would have an internal timer of `7`.

A lanternfish that creates a new fish resets its timer to `6`, _not `7`_ (because `0` is included as a valid timer value). The new lanternfish starts with an internal timer of `8` and does not start counting down until the next day.

Realizing what you're trying to do, the submarine automatically produces a list of the ages of several hundred nearby lanternfish (your puzzle input). For example, suppose you were given the following list:

```
3,4,3,1,2
```

This list means that the first fish has an internal timer of `3`, the second fish has an internal timer of `4`, and so on until the fifth fish, which has an internal timer of `2`. Simulating these fish over several days would proceed as follows:

```
Initial state: 3,4,3,1,2
After  1 day:  2,3,2,0,1
After  2 days: 1,2,1,6,0,8
After  3 days: 0,1,0,5,6,7,8
After  4 days: 6,0,6,4,5,6,7,8,8
After  5 days: 5,6,5,3,4,5,6,7,7,8
After  6 days: 4,5,4,2,3,4,5,6,6,7
After  7 days: 3,4,3,1,2,3,4,5,5,6
After  8 days: 2,3,2,0,1,2,3,4,4,5
After  9 days: 1,2,1,6,0,1,2,3,3,4,8
After 10 days: 0,1,0,5,6,0,1,2,2,3,7,8
After 11 days: 6,0,6,4,5,6,0,1,1,2,6,7,8,8,8
After 12 days: 5,6,5,3,4,5,6,0,0,1,5,6,7,7,7,8,8
After 13 days: 4,5,4,2,3,4,5,6,6,0,4,5,6,6,6,7,7,8,8
After 14 days: 3,4,3,1,2,3,4,5,5,6,3,4,5,5,5,6,6,7,7,8
After 15 days: 2,3,2,0,1,2,3,4,4,5,2,3,4,4,4,5,5,6,6,7
After 16 days: 1,2,1,6,0,1,2,3,3,4,1,2,3,3,3,4,4,5,5,6,8
After 17 days: 0,1,0,5,6,0,1,2,2,3,0,1,2,2,2,3,3,4,4,5,7,8
After 18 days: 6,0,6,4,5,6,0,1,1,2,6,0,1,1,1,2,2,3,3,4,6,7,8,8,8,8
```

Each day, a `0` becomes a `6` and adds a new `8` to the end of the list, while each other number decreases by 1 if it was present at the start of the day.

In this example, after 18 days, there are a total of `26` fish. After 80 days, there would be a total of `_5934_`.

Find a way to simulate lanternfish. _How many lanternfish would there be after 80 days?_

_Using the input file `input.txt`, the result should be 345387._

In [2]:
# IMPORTANT: Set this to the correct path for you!
INPUT_FILE = "data/input.txt"

## Baseline solution

Motivated by the visual explanation given in the problem statement, we can simulate the growth of the population of lanternship:

In [7]:
with open(INPUT_FILE, "r") as f:
    fish_pop = [int(num) for num in f.readline().split(",")]

for _ in range(80):
    new = 0
    for idx in range(len(fish_pop)):
        fish_pop[idx] -= 1
        if fish_pop[idx] < 0:
            new += 1
            fish_pop[idx] = 6
    fish_pop += [8] * new

print(len(fish_pop))

345387


The above is a first attempt at solving the problem, and I would like to take this example code to point out a few alternative ways of doing some common tasks.

The first thing I'd like to point out is that the simulation doesn't care about the number of cycles we already went through, and that is why the outer `for` loop has a `_` as the variable name: `for _ in range(80)`.
This is an idiomatic way of saying that all iterations of the loop do exactly the same thing.

Next, a `for` loop of the form `for ... in range(len(...))` is generally _not_ what you want to do.
In this particular case, we can improve our `for` loop a bit more by using the built-in `enumerate`:

In [8]:
with open(INPUT_FILE, "r") as f:
    fish_pop = [int(num) for num in f.readline().split(",")]

for _ in range(80):
    new = 0
    for idx, state in enumerate(fish_pop):
        fish_pop[idx] -= 1
        if state == 0:
            new += 1
            fish_pop[idx] = 6
    fish_pop += [8] * new

print(len(fish_pop))

345387


## Part 2 problem statement

Suppose the lanternfish live forever and have unlimited food and space. Would they take over the entire ocean?

After 256 days in the example above, there would be a total of `_26984457539_` lanternfish!

_How many lanternfish would there be after 256 days?_

_Using the input file `input.txt`, the result should be 1574445493136._

## Exponential space complexity

The part 1 problem statement mentions that the population of lanternfish grows _exponentially_, and programmers generally don't like things that grow exponentially: if an algorithm's space – or time – complexity is exponential, that means you will run out of space – or time – astoundingly quickly!

Thus, things won't go well for us if we try running the same code as above, but for `256` days – even though `256` is such a “small” number!

Therefore, we need to reduce the space-complexity of our solution.

To do that, we can realise that we don't need to have a single integer for each fish, we can very well just keep track of how many fish are in each state of their life cycles!

Let's use a list for that, where `fish_pop[idx]` contains the number of fish with internal timer set to `idx`:

In [11]:
with open(INPUT_FILE, "r") as f:
    fish_input = [int(num) for num in f.readline().split(",")]

# fish_pop = [fish_input.count(idx) for idx in range(9)]

fish_pop = [0] * 9
for fish in fish_input:
    fish_pop[fish] += 1

for _ in range(256):
    zeros = fish_pop[0]
    for timer in range(1, 9):
        fish_pop[timer - 1] = fish_pop[timer]
    fish_pop[8] = zeros
    fish_pop[6] += zeros
    
print(sum(fish_pop))

[143094129622, 173596785972, 178948425837, 195663384795, 224807310351, 222191335428, 275179254454, 118788062918, 150307958118]
1682576647495


This solution is very fast and its space complexity went down from $O(2^n)$ to $O(1)$ (from exponential to constant), where $n$ is the number of iterations.
That's one heck of an improvement!

## Fighting asymmetries

It's only day 6, and you are probably tired of hearing me talk about asymmetries, but the code above shows a clearly asymmetry: index `8` has to be handled separately.

Why is that?
Because index `8` would get the value from index `0`, but that value is written over as soon as the `timer` loop starts, so we need to store the value at index `0` for later.

Now, you might be wondering whether or not you could write the `for` loop as `for timer in range(9)`.
After all, when `timer = 0`, we have `fish_pop[-1] = fish_pop[0]`, and an index of `-1` means “the last element”, which is `fish_pop[8]` in this case.
This _could_ work, except that now we are writing over `fish_pop[8]` before we update the value in `fish_pop[7]`, so we have this cyclical issue...

But what if we exploit the cyclical nature of the updates?
Using slices, we should be able to cycle the data around:

In [16]:
with open(INPUT_FILE, "r") as f:
    fish_input = [int(num) for num in f.readline().split(",")]

fish_pop = [0] * 9
for fish in fish_input:
    fish_pop[fish] += 1

for _ in range(256):
    fish_pop = fish_pop[1:] + [fish_pop[0]]
    fish_pop[6] += fish_pop[-1]
    
print(sum(fish_pop))

1574445493136


We just need to be careful about how we increment the count of fish that have reproduced.
After the cycle it's `fish_pop[6] += fish_pop[-1]`, because the fish that had a timer at `0` were just moved to the end.
If we do it _before_ the cycle, it becomes `fish_pop[7] += fish_pop[0]`:

In [17]:
with open(INPUT_FILE, "r") as f:
    fish_input = [int(num) for num in f.readline().split(",")]

fish_pop = [0] * 9
for fish in fish_input:
    fish_pop[fish] += 1

for _ in range(256):
    fish_pop[7] += fish_pop[0]
    fish_pop = fish_pop[1:] + [fish_pop[0]]
    
print(sum(fish_pop))

1574445493136


## Stationary data and wrapping with `%`

We are exploiting the cyclicity of the updates, but we keep moving data around.
Even though it sounds conceptually simple, slicing a list produces a new every time, and moving the data around can be painful for Python.

The price we are paying isn't a price that we can't afford, but sometimes we need to find ways of moving less data around.
For our case, instead of rotating the list, we can keep create a `pointer` variable that keeps track of the position of the list that should be _seen_ as the beginning of the list.

Finally, we can use “pointer arithmetics” to figure out what position should get incremented by what position.

Take a look at the following excerpt from the previous solution:

```py
for _ in range(256):
    fish_pop[7] += fish_pop[0]               # updating births
    fish_pop = fish_pop[1:] + [fish_pop[0]]  # cycling
```

The cycling operation is now simulated by incrementing the pointer, which points at the number of fish that will reproduce next.
The operation of updating births is taking the fish at position `0` (the fish that will reproduce next) and adding them to position `7`, so it's as if the pointer were `pointer = 0` and the line of code was `fish_pop[pointer + 7] = fish_pop[pointer]`.

However, this won't work:

In [20]:
with open(INPUT_FILE, "r") as f:
    fish_input = [int(num) for num in f.readline().split(",")]

fish_pop = [0] * 9
for fish in fish_input:
    fish_pop[fish] += 1

pointer = 0
for _ in range(256):
    fish_pop[pointer + 7] += fish_pop[pointer]
    pointer += 1
    
print(sum(fish_pop))

IndexError: list index out of range

This doesn't work because the variable `pointer` quickly grows too large to be able to index into the list.
To fix this, and to simulate the cyclicity/wrapping around, we use the modulo operator `%`.

By using the modulo operator with the length of the list, we can keep incrementing the `pointer` and _always_ have it point to a valid index.

Here is a simple demonstration of that:

In [21]:
s = "abcde"
for idx in range(20):
    print(s[idx % len(s)], end=" ")

a b c d e a b c d e a b c d e a b c d e 

Notice how `idx` grew up to `19`, but the `%` ensured we kept cycling through the elements of `s`.

For our problem, we need to use `%` to make sure that our “pointer arithmetics” always produce legal indices:

In [22]:
with open(INPUT_FILE, "r") as f:
    fish_input = [int(num) for num in f.readline().split(",")]

fish_pop = [0] * 9
for fish in fish_input:
    fish_pop[fish] += 1

pointer = 0
for _ in range(256):
    fish_pop[(pointer + 7) % 9] += fish_pop[pointer]
    pointer = (pointer + 1) % 9
    
print(sum(fish_pop))

1574445493136


## Proper cycling

To close off this chapter of cycling without moving data around, I will introduce you to the `itertools.cycle` function, which does exactly what I demonstrated with `s` above:

In [23]:
from itertools import cycle

s = "abcde"
for char, _ in zip(cycle(s), range(20)):
    print(char, end=" ")

a b c d e a b c d e a b c d e a b c d e 

The usage of `cycle` allows us to continuously cycle through the elements of `s`.
We had to `zip` that with `range(20)` to print `20` characters, otherwise we would keep on going.

Alternatively, we can make use of yet another `itertools` tool that we can use to slice iterators – even if they are infinite! – as shown here:

In [24]:
from itertools import cycle, islice

s = "abcde"
for char in islice(cycle(s), 20):
    print(char, end=" ")

a b c d e a b c d e a b c d e a b c d e 

Using `cycle` and `islice`, we don't even need to manage the `pointer` ourselves:

In [26]:
from itertools import cycle, islice

with open(INPUT_FILE, "r") as f:
    fish_input = [int(num) for num in f.readline().split(",")]

fish_pop = [0] * 9
for fish in fish_input:
    fish_pop[fish] += 1

for pointer in islice(cycle(range(9)), 256):
    fish_pop[(pointer + 7) % 9] += fish_pop[pointer]
    
print(sum(fish_pop))

1574445493136


## Do not count by hand

Ok, I lied.
There is one more thing I would like to mention in this family of solutions:
with Python, there really is not reason for you to count things by hand.
You've met `collections.Counter`, so why not use it to count the initial fish states?

In [32]:
from collections import Counter
from itertools import cycle, islice

with open(INPUT_FILE, "r") as f:
    fish_pop = Counter(int(num) for num in f.readline().split(","))

for pointer in islice(cycle(range(9)), 256):
    fish_pop[(pointer + 7) % 9] += fish_pop[pointer]
    
print(fish_pop.total())  # Counter.total new in Python 3.10.

1574445493136


## Part 3 problem statement

This part 3 was put forward by a user of the exclusive Discord area for the bootcamp participants:

Compute the total number of fish available after $10^{12}$ cycles, modulo 1000000007.

Our $O(n)$ method will _not_ be able to cope with this.
I invite you to give it a go!

So, we have to figure out a faster method...

## What about time efficiency?

We have explored a very space-efficient solution.
It's as efficient as they come.

But what about time efficiency?

Can we build a solution that is better than $O(n)$, where $n$ is the number of cycles we need to go through?

As it turns out, we can!

Have you ever heard about the [Fibonacci sequence](https://en.wikipedia.org/wiki/Fibonacci_number)?
Like it or not, I am sure you have.

Much like the backstory to this problem includes lanternfish reproducing, the Fibonacci sequence is sometimes defined as the rate with which rabbits reproduce (in an idealised scenario).
Quoting the [Wikipedia article](https://en.wikipedia.org/wiki/Fibonacci_number#History):

 > “Fibonacci considers the growth of an idealized (biologically unrealistic) [rabbit](https://en.wikipedia.org/wiki/Rabbit "Rabbit") population, assuming that:
 > a newly born breeding pair of rabbits are put in a field;
 > each breeding pair mates at the age of one month, and at the end of their second month they always produce another pair of rabbits;
 > and rabbits never die, but continue breeding forever.
 > Fibonacci posed the puzzle: how many pairs will there be in one year?
 >
 > -   At the end of the first month, they mate, but there is still only 1 pair.
 > -   At the end of the second month they produce a new pair, so there are 2 pairs in the field.
 > -   At the end of the third month, the original pair produce a second pair, but the second pair only mate without breeding, so there are 3 pairs in all.
 > -   At the end of the fourth month, the original pair has produced yet another new pair, and the pair born two months ago also produces their first pair, making 5 pairs.”

As you can see, there are some similarities in the backstories associated with these two sequences.
In fact, the problem of computing terms of this sequence is mathematically equivalent to computing terms of the Fibonacci sequence.
As the Internet will tell you, you _can_ compute terms of the Fibonacci sequence and thus so can you do it for this sequence.

(The discussion that follows assumes you are familiar with matrix multiplication.)

We start by building a matrix and a vector:

 - we build a matrix $A$ that encodes how the counts of fish are updated from one step to the other;
 - we represent the original counts of fish in a column vector $v$;

At this point, $Av$ computes the fish count after one cycle; $A^2v$ computes the fish after two cycles; and $A^nv$ computes the fish counts after $n$ cycles.
If one uses [exponentiation by squaring](https://en.wikipedia.org/wiki/Exponentiation_by_squaring) to compute $A^n$, then we can compute $A^n$ in $O(\log n)$ time!

## Building the matrix

Let's use $x_i$ to represent the nine positions of the vector we have been working with, that keeps track of the fish counts, where $i$ matches the index in the list:

$$
x = \begin{bmatrix}
x_0 \\
x_1 \\
x_2 \\
x_3 \\
x_4 \\
x_5 \\
x_6 \\
x_7 \\
x_8\end{bmatrix}
$$

$x_0$ represents the number of fish that are _ready_ to reproduce, $x_1$ represents the number of fish that will reproduce after one cycle, etc.

After one cycle, the new vector is updated to look like this:

$$
x' = \begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
x_4 \\
x_5 \\
x_6 \\
x_7 + x_0 \\
x_8 \\
x_0\end{bmatrix}
$$

What we need to do is build a matrix $A$ such that $x' = Ax$.
Thankfully, the matrix isn't too complicated:

$$
\begin{bmatrix}
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\ 
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\ 
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\ 
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0\\ 
1 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\ 
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\\ 
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0
\end{bmatrix}
$$

Notice how there is a diagonal of 1s going down the matrix, and there are two loose 1s to the left, near the bottom.

In Python, we can represent this matrix by

In [33]:
A = [
    [0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0],
]

Ideally, we would use something like `numpy` to represent this matrix and to do the matrix multiplications.
Because I don't want to assume you have any particular dependencies installed, we can take this opportunity to exercise our coding muscles: matrix multiplication is a classical algorithm everyone should implement once in their lifetime:

In [44]:
def matmul(A, B):
    """Perform matrix multiplication between A and B."""
    return [[sum(a * b for a, b in zip(row, col)) for col in zip(*B)]for row in A]

## Exponentiation by squaring

The method of exponentiation by squaring is a clever method to speed up exponentiation.

The operation $A^n$ takes $O(n)$ time if you compute it as $A \times A \times A \times \cdots \times A$.
Instead, we decompose $n$ into its binary expansion, and use that information to compute $A^n$ in $O(\log n)$.
(You can refer to [this link](https://en.wikipedia.org/wiki/Exponentiation_by_squaring#Basic_method) for a brief overview of how this works.)

We can implement it like so:

In [73]:
def fast_power(A, n):
    if n == 1:
        return A
    elif n % 2:
        return A * fast_power(A * A, n // 2)
    else:
        return fast_power(A * A, n // 2)

In [74]:
fast_power(2, 17)

131072

In [75]:
pow(2, 17)

131072

The (recursive) definition of `fast_power` above isn't as complete as it could be because it doesn't handle all edge cases (what about `n == 0` or negative `n`?) but is enough for our purposes.

We just need to make it a _bit_ more general to accept an optional argument that implements multiplication for the parameter `A`:

In [96]:
from operator import mul

def fast_power(A, n, mul=mul):
    if n == 1:
        return A
    elif n % 2:
        return mul(A, fast_power(mul(A, A), n // 2, mul))
    else:
        return fast_power(mul(A, A), n // 2, mul)

In [97]:
fast_power(2, 17)

131072

## Computing the total number of fish with matrix multiplication

Now that we know how to exponentiate $A$, we can put everything together and make sure it's working:

In [106]:
from collections import Counter
from itertools import cycle, islice

with open(INPUT_FILE, "r") as f:
    fish_pop = Counter(int(num) for num in f.readline().split(","))

vec = [[fish_pop[i]] for i in range(9)]  # [[x0], [x1], ...] for a column vector.
A256 = fast_power(A, 256, matmul)
end = matmul(A256, vec)
print(sum(v[0] for v in end))

1574445493136


## Power-modulo

The reason that part 3 asks for the result modulo 1000000007 is because numbers will **explode** if we don't use a modulo to keep them “small”.

In order to do the modulo operation, we can just modify our function `matmul`:

In [107]:
def matmul(A, B, mod=None):
    """Perform matrix multiplication between A and B, optionally modulo mod."""
    res = [[sum(a * b for a, b in zip(row, col)) for col in zip(*B)]for row in A]
    if mod is not None:
        res = [[val % mod for val in row] for row in res]
    return res

Above, we checked that our method computes the correct solution for 256 cycles, and the correct answer was 1574445493136, so the correct answer is 93136 if we compute it modulo 100000.
Let's see if it adds up with our new function `matmul`:

In [110]:
A256 = fast_power(A, 256, lambda A, B: matmul(A, B, 100000))
end = matmul(A256, vec, 100000)
print(sum(v[0] for v in end) % 100000)

93136


Finally, we can check if our fast method can cope with the $10^{12}$ (that's 1,000,000,000,000) cycles:

In [111]:
mod = 1000000007
A_ = fast_power(A, 10 ** 12, lambda A, B: matmul(A, B, mod))
end = matmul(A_, vec, mod)
print(sum(v[0] for v in end) % mod)

53123166


Well, I have _no_ idea if this is correct 😂
But I have a strong suspicion it _is_, and it was computed blazingly fast!
I'm _still_ waiting for the linear method above to finish...

## Conclusion

In order to write an algorithm that was fast, we drew inspiration from a similar problem: the Fibonacci sequence.
This is _extremely_ important: the more problems you solve, the more techniques, algorithms, tricks, etc, you learn.
Ultimately, this is one of the key characteristics of a good problem-solver: experience.

If you have any questions, suggestions, remarks, recommendations, corrections, or anything else, you can reach out to me [on Twitter](https://twitter.com/mathsppblog) or via email to rodrigo at mathspp dot com.